In [1]:
import pandas as pd
import nltk
from nltk.tokenize import TweetTokenizer
from os.path import join

### Data import

In [2]:
sample = pd.read_json(join('..', 'data', 'ner', 'sample_100_pages_names_tokens.json'))

In [3]:
sample.reset_index(drop=True, inplace=True)

In [4]:
sample.columns

Index(['categories', 'date', 'description', 'files', 'id', 'location', 'title',
       'matched_name', 'matched_category', 'matched_city', 'matched_address',
       'matched_title', 'matched_title_2', 'matched_title_str',
       'matched_title_2_str'],
      dtype='object')

### Stop words

In [5]:
stopwords_sample = pd.read_json(join('..', 'data', 'index', 'stopwords-bg.json'))[259:]

In [6]:
short_stop_word = stopwords_sample[stopwords_sample[0].apply(lambda x: len(x) <= 3)][0]

### Tokenize search columns

In [7]:
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True)

In [8]:
columns_for_index = ['matched_name', 'matched_title_str', 'matched_title_2_str', 'location']



sample[columns_for_index] = sample[columns_for_index].fillna("")

for col in columns_for_index:
    sample[col + '_tokens'] = sample[col].apply(tokenizer.tokenize)

sample['tokens'] = (sample['matched_name_tokens'] +
                sample['matched_title_str_tokens'] +
                sample['matched_title_2_str_tokens'] +
                sample['location_tokens'])

### TF-IDF

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
sample['search_tokens_text'] = sample['tokens'].apply(lambda x: ' '.join(x))

In [11]:
tfidf_vectorizer = TfidfVectorizer(stop_words=short_stop_word.tolist())
tfidf_vectorizer.fit_transform(sample['search_tokens_text'])

<3000x3280 sparse matrix of type '<class 'numpy.float64'>'
	with 23153 stored elements in Compressed Sparse Row format>

### Cosine similarity on corpus

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
train_corpus = tfidf_vectorizer.transform(sample['search_tokens_text'])

Look for similarity betweet each document

In [14]:
duplicates = cosine_similarity(train_corpus, train_corpus)

In [15]:
duplicates

array([[1.        , 0.01394738, 0.00175835, ..., 0.00559053, 0.00738054,
        0.03803982],
       [0.01394738, 1.        , 0.00453168, ..., 0.02399727, 0.01902139,
        0.00507922],
       [0.00175835, 0.00453168, 1.        , ..., 0.00181643, 0.11174247,
        0.01313839],
       ...,
       [0.00559053, 0.02399727, 0.00181643, ..., 1.        , 0.00762435,
        0.06245791],
       [0.00738054, 0.01902139, 0.11174247, ..., 0.00762435, 1.        ,
        0.0167978 ],
       [0.03803982, 0.00507922, 0.01313839, ..., 0.06245791, 0.0167978 ,
        1.        ]])

Filter pairs with similarity >= 0.9

In [16]:
dup_pairs = []

for i in range(len(sample)):
    for j in range(len(sample)):
        if duplicates[i, j] >= 0.9 and i < j:
            dup_pairs.append((i, j))

In [17]:
len(dup_pairs)

1169

In [18]:
dup_pairs[:20]

[(3, 2605),
 (4, 621),
 (4, 1395),
 (4, 1952),
 (4, 2490),
 (10, 1433),
 (14, 2483),
 (16, 2388),
 (17, 1366),
 (17, 2794),
 (19, 1236),
 (30, 237),
 (30, 2413),
 (32, 1902),
 (33, 2261),
 (33, 2729),
 (36, 107),
 (36, 2182),
 (36, 2936),
 (45, 133)]

In [19]:
for doc1, doc2 in dup_pairs[:20]:
    print(sample.iloc[doc1][['title', 'description', 'location']])
    print(sample.iloc[doc2][['title', 'description', 'location']])
    print()

title                Масово пушене в заведение Маки, град Сливен
description    На 24.11.2017, 19:30 часа, от 20 маси на 6 се ...
location                  град Сливен, бул. Цар Освободител 15 а
Name: 3, dtype: object
title                       Пушене в заведение Маки, град Сливен
description    В заведението се пуши непрекъснато. * Сигналът...
location                   град Сливен, ул. Цар Освободител 17 А
Name: 2605, dtype: object

title          Нарушаване на забраната за тютюнопушене в пица...
description    В Пицария Ветрило се пуши необезпокоявано и съ...
location                         град София, ул Орехова гора 42А
Name: 4, dtype: object
title          Пушене в ресторант-пицария Ветрило, град София...
description    име обект:  Ветрило вид обект: ресторант пицар...
location       град София, жк. Стрелбище, ул. Орехова гора № 42А
Name: 621, dtype: object

title          Нарушаване на забраната за тютюнопушене в пица...
description    В Пицария Ветрило се пуши необезпокоявано

Look for similar documnet for a given document in the corpus

In [20]:
def query_group(df_ix):
    group = cosine_similarity(train_corpus[df_ix], train_corpus)[0]
    
    dup_tuples = []
    dub_ids = []

    for j in range(3000):
        if group[j] >= 0.9:
            dup_tuples.append(j)
            
    for ix in dup_tuples:
        dub_ids.append(sample.iloc[ix]['id'])
            
    return dub_ids

In [21]:
similar_ids = query_group(2121)
similar_ids

[9085, 7736, 8076, 7040]

In [22]:
sample[sample.id.isin(similar_ids)][['title', 'description', 'location']]

,title,description,location
271,"Пушене в ресторант Ветрило, град София (мобиле...",име обект: Ветрило вид обект: ресторант град: ...,"град София, ж.к. Стрелбище, ул. Орехова гора №..."
1308,Нарушаване на забраната за пушене в ресторант ...,На 27/01/2015 в 12.30ч. на обяд се пушеше своб...,"град София, ул.Орехова гора 42А"
1675,"Сигнал за цигарен дим в ресторант ""Ветрило"", ж...","Здравейте, Искам да подам сигнал за ежедневно ...","София, ул. Орехова гора 42А"
2121,"Систематично пушене на затворено в ресторант ""...","В ресторант Ветрило в София, Стрелбище, ул. Ор...","София, ул.Орехова Гора 42А"


In [23]:
similar_ids_2 = query_group(213)

In [24]:
sample[sample.id.isin(similar_ids_2)][['title', 'description', 'location']]

,title,description,location
213,Неспазване на забраната за тютюнопушене н пица...,В ресторанта се пуши непрекъснато. * Сигналът ...,"град София, ул. Солун 43"
679,"Масово тютюнопушене в пицария ""Ветрило"", грд С...","В пицария ""Ветрило"" на ул. ""Солун"" в ж.к. ""Бор...","град София, ул. ""Солун"", срещу бл. 43"
2077,"Пушене в пицария ""Ветрило"", град София","Безкрайно съм възмутен от ситуацията, в която ...","град София, ул. Солун, с/у блок 43"


### Soft duplicates

In [25]:
def has_more(tuples, ix):
    return any(filter(lambda x: x[0] == ix or x[1] == ix, tuples))

In [26]:
has_more(dup_pairs, 70)

True

In [27]:
def find_first(tuples, ix):
    return next((x[0] for x in tuples if x[1] == ix), None)

In [28]:
# find_first(dup_pairs, 2490)

In [29]:
def get_all_dubs(tuples, ix, group):
    if not has_more(tuples, ix):
        group.append(ix)
        return
    else:
        for i, (ix1, ix2) in enumerate(tuples):
            if ix1 == ix:
                del tuples[i]
                group.append(ix1)
                get_all_dubs(tuples, ix1, group)
                get_all_dubs(tuples, ix2, group)
            elif ix2 == ix:
                del tuples[i]
                group.append(ix2)
                get_all_dubs(tuples, ix1, group)
                get_all_dubs(tuples, ix2, group)
                
    return set(group)

In [30]:
from copy import deepcopy

In [31]:
def find_number_of_duplicates(tuples, ix):
    tuples_copy = deepcopy(tuples)
    return get_all_dubs(tuples_copy, ix, []) 

In [32]:
# no pair (4, 271) exists but there is pair (271, 621) - soft duplicates
# no pairs (679, x) or (2077, x) - but there are pairs (213, 629) and (213, 2077)

gr = find_number_of_duplicates(dup_pairs, 679)  
gr

{213, 679, 2077}

In [33]:
sample.iloc[list(gr)][['title', 'location']]

,title,location
2077,"Пушене в пицария ""Ветрило"", град София","град София, ул. Солун, с/у блок 43"
213,Неспазване на забраната за тютюнопушене н пица...,"град София, ул. Солун 43"
679,"Масово тютюнопушене в пицария ""Ветрило"", грд С...","град София, ул. ""Солун"", срещу бл. 43"


In [34]:
gr = find_number_of_duplicates(dup_pairs, 1675)  
gr

{4, 271, 621, 1308, 1395, 1675, 1952, 2121, 2490}

In [35]:
sample.iloc[list(gr)][['title', 'location']]

,title,location
1952,"пушене в пицария ресторант ""Ветрило"", град Соф...","град София, Ул. Орехова гора 42А"
4,Нарушаване на забраната за тютюнопушене в пица...,"град София, ул Орехова гора 42А"
2121,"Систематично пушене на затворено в ресторант ""...","София, ул.Орехова Гора 42А"
1675,"Сигнал за цигарен дим в ресторант ""Ветрило"", ж...","София, ул. Орехова гора 42А"
621,"Пушене в ресторант-пицария Ветрило, град София...","град София, жк. Стрелбище, ул. Орехова гора № 42А"
271,"Пушене в ресторант Ветрило, град София (мобиле...","град София, ж.к. Стрелбище, ул. Орехова гора №..."
1395,"пушене в Ресторант пицария Ветрило , София (мо...","София, Кв . Стрелбище, Ул. Орехова гора 42а"
2490,"Пушене в Пицария Ветрило, град София","град София, жк Стрелбище, ул. ""Орехова гора"" ..."
1308,Нарушаване на забраната за пушене в ресторант ...,"град София, ул.Орехова гора 42А"
